## Deliverable 2. Create a Customer Travel Destinations Map.

In [1]:
# Dependencies and Setup
import pandas as pd
import requests
import gmaps

# Import API key
from config import g_key

# Configure gmaps API key
gmaps.configure(api_key=g_key)

In [2]:
# 1. Import the WeatherPy_database.csv file. 
city_data_df = pd.read_csv("../Weather_Database/WeatherPy_database.csv")
city_data_df.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Current Description
0,0,Punta Arenas,-53.1500,-70.9167,42.91,93,100,10.36,CL,light rain
1,1,Rikitea,-23.1203,-134.9692,71.20,73,99,21.54,PF,overcast clouds
2,2,Geraldton,-28.7667,114.6000,57.61,88,100,18.41,AU,overcast clouds
3,3,Sept-Iles,50.2001,-66.3821,60.80,66,20,12.66,CA,few clouds
4,4,Upernavik,72.7868,-56.1549,45.57,95,100,28.86,GL,light rain


In [3]:
# 2. Prompt the user to enter minimum and maximum temperature criteria 
# Ask the customer to add a minimum and maximum temperature value.
min_temp = float(input("What is the minimum temperature you would like for your trip? "))
max_temp = float(input("What is the maximum temperature you would like for your trip? "))

What is the minimum temperature you would like for your trip? 75
What is the maximum temperature you would like for your trip? 90


In [4]:
# 3. Filter the city_data_df DataFrame using the input statements to create a new DataFrame using the loc method.
# Filter the dataset to find the cities that fit the criteria.
preferred_cities_df = city_data_df.loc[(city_data_df["Max Temp"] <= max_temp) & \
                                       (city_data_df["Max Temp"] >= min_temp)]
preferred_cities_df.head(10)

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Current Description
14,14,Puerto Ayora,-0.7393,-90.3518,76.96,86,89,8.99,EC,overcast clouds
17,17,Torbay,47.6666,-52.7314,79.29,77,75,13.80,CA,broken clouds
19,19,Butaritari,3.0707,172.7902,81.07,79,80,16.13,KI,light rain
20,20,Mangrol,21.1167,70.1167,80.96,85,8,8.61,IN,clear sky
21,21,Atuona,-9.8000,-139.0333,76.75,75,65,22.37,PF,broken clouds
22,22,Arraial Do Cabo,-22.9661,-42.0278,80.56,50,0,20.71,BR,clear sky
35,35,Kapaa,22.0752,-159.3190,82.09,72,20,11.50,US,few clouds
36,36,Albany,42.6001,-73.9662,78.62,43,0,3.00,US,clear sky
41,41,Sao Filipe,14.8961,-24.4956,78.91,86,77,15.01,CV,light rain
47,47,Hilo,19.7297,-155.0900,87.28,74,0,5.75,US,clear sky


In [5]:
# 4a. Determine if there are any empty rows.
preferred_cities_df.count()

City_ID                231
City                   231
Lat                    231
Lng                    231
Max Temp               231
Humidity               231
Cloudiness             231
Wind Speed             231
Country                231
Current Description    231
dtype: int64

In [6]:
# 4b. Drop any empty rows and create a new DataFrame that doesn’t have empty rows.
clean_df = preferred_cities_df.dropna(axis=0)

In [7]:
# 5a. Create DataFrame called hotel_df to store hotel names along with city, country, max temp, and coordinates.
hotel_df = clean_df[["City", "Country", "Max Temp", "Current Description", "Lat", "Lng"]].copy()

# 5b. Create a new column "Hotel Name"
hotel_df["Hotel Name"] = ""
hotel_df.head(10)

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
14,Puerto Ayora,EC,76.96,overcast clouds,-0.7393,-90.3518,
17,Torbay,CA,79.29,broken clouds,47.6666,-52.7314,
19,Butaritari,KI,81.07,light rain,3.0707,172.7902,
20,Mangrol,IN,80.96,clear sky,21.1167,70.1167,
21,Atuona,PF,76.75,broken clouds,-9.8000,-139.0333,
22,Arraial Do Cabo,BR,80.56,clear sky,-22.9661,-42.0278,
35,Kapaa,US,82.09,few clouds,22.0752,-159.3190,
36,Albany,US,78.62,clear sky,42.6001,-73.9662,
41,Sao Filipe,CV,78.91,light rain,14.8961,-24.4956,
47,Hilo,US,87.28,clear sky,19.7297,-155.0900,


In [8]:
# 6a. Set parameters to search for hotels with 5000 meters.
params = {
    "radius": 5000,
    "type": "lodging",
    "key": g_key
}

# 6b. Iterate through the hotel DataFrame.
for index, row in hotel_df.iterrows():
    # 6c. Get latitude and longitude from DataFrame
    lat = row["Lat"]
    lng = row["Lng"]
    params["location"] = f"{lat},{lng}"
    
    # 6d. Set up the base URL for the Google Directions API to get JSON data.
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

    # 6e. Make request and retrieve the JSON data from the search. 
    hotels = requests.get(base_url, params=params).json()
    
    # 6f. Get the first hotel from the results and store the name, if a hotel isn't found skip the city.
    try:
        hotel_df.loc[index, "Hotel Name"] = hotels["results"][0]["name"]
    except (IndexError):
        print("Hotel not found... skipping.")
        

Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.


In [9]:
# 7. Drop the rows where there is no Hotel Name.
clean_hotel_df = hotel_df.dropna()

In [10]:
# 8a. Create the output File (CSV)
output_data_file = "WeatherPy_vacation.csv"
# 8b. Export the City_Data into a csv
clean_hotel_df.to_csv(output_data_file, index_label="City_ID")

In [11]:
# 9. Using the template add city name, the country code, the weather description and maximum temperature for the city.
info_box_template = """
<dl>
<dt>Hotel Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
<dt>Current Description</dt><dd>{Current Description}</dd><dt> and </dt><dd>{Max Temp} °F</dd>
</dl>
"""

# 10a. Get the data from each row and add it to the formatting template and store the data in a list.
hotel_info = [info_box_template.format(**row) for index, row in clean_hotel_df.iterrows()]

# 10b. Get the latitude and longitude from each row and store in a new DataFrame.
locations = clean_hotel_df[["Lat", "Lng"]]

In [12]:
# 11a. Add a marker layer for each city to the map. 
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)
marker_layer = gmaps.marker_layer(locations,info_box_content=hotel_info)
fig.add_layer(marker_layer)

# 11b. Display the figure
fig

Figure(layout=FigureLayout(height='420px'))